# SQLAlchemy

In [1]:
import datetime
import sqlalchemy as sql
from sqlalchemy.ext.declarative import declarative_base
 
Base = declarative_base()

# Schema

class Program(Base):
    """ tests """
    __tablename__ = 'Programs'
    id = sql.Column(sql.String(40), primary_key=True)
    date = sql.Column(sql.DateTime, default=datetime.datetime.utcnow)
    cli = sql.Column(sql.Text, nullable=False)
    contents = sql.Column(sql.Text, nullable=False)

    def __repr__(self):
        return self.id

    
class Device(Base):
    """ devices """
    __tablename__ = 'Devices'
    id = sql.Column(sql.Integer, primary_key=True)
    hostname = sql.Column(sql.String(63), nullable=False)  # RFC 1035
    platform = sql.Column(sql.Text, nullable=False)
    device = sql.Column(sql.Text, nullable=False)
    __table_args__ = (sql.UniqueConstraint('hostname', 'platform', 'device', name='_uid'),)
    
    def __repr__(self):
        return "Host: {self.hostname}, Platform: {self.platform}, Device: {self.device}".format(**vars())
    
    
class Params(Base):
    """ params """
    __tablename__ = "Parameters"
    id = sql.Column(sql.Integer, primary_key=True)
    gsize_x = sql.Column(sql.Integer, nullable=False)
    gsize_y = sql.Column(sql.Integer, nullable=False)
    gsize_z = sql.Column(sql.Integer, nullable=False)
    lsize_x = sql.Column(sql.Integer, nullable=False)
    lsize_y = sql.Column(sql.Integer, nullable=False)
    lsize_z = sql.Column(sql.Integer, nullable=False)


class Result(Base):
    __tablename__ = "Results"
    id = sql.Column(sql.Integer, primary_key=True)
    program_id = sql.Column(sql.String(40), sql.ForeignKey("Programs.id"))
    device_id = sql.Column(sql.Integer, sql.ForeignKey("Devices.id"))
    date = sql.Column(sql.DateTime, default=datetime.datetime.utcnow)
    cli = sql.Column(sql.Text, nullable=False)
    status = sql.Column(sql.Integer, nullable=False)
    runtime = sql.Column(sql.Float, nullable=False)
    stdout = sql.Column(sql.Text, nullable=False)
    stderr = sql.Column(sql.Text, nullable=False)

In [2]:
!rm -f sqlalchemy.db

# create the engine
engine = sql.create_engine('sqlite:///sqlalchemy.db') 
# create all tables in the engine
Base.metadata.create_all(engine)

In [3]:
# bind the engine to the metadata of the Base class so that the declaratives can be accessed
Base.metadata.bind = engine
make_session = sql.orm.sessionmaker(bind=engine)

In [4]:
def program1():
    return Program(
    id="adc83b19e793491b1c6ea0fd8b46cd9f32e592fc", cli="./foo prog1", contents="int main() {}")
def program1_key_dupe():
    return Program(
    id="adc83b19e793491b1c6ea0fd8b46cd9f32e592fc", cli="./foo bar", contents="int main() {}")
def program2():
    return Program(
    id="f1d2d2f924e986ac86fdf7b36c94bcdf32beec15", cli="./foo prog2", contents="int main(int argc, char **argv) {}")

In [5]:
# if we don't call session.commit(), the database isn't modified
session = make_session()
session.add(program1())
session.add(program2())
session.close()

In [6]:
# if we call session.rollback(), changes are ignored
session = make_session()
session.add(program1())
session.add(program2())
session.rollback()
session.close()

In [7]:
# add the entries
session = make_session()
try:
    session.add(program1())
    session.add(program2())
    session.commit()
except sql.exc.IntegrityError:
    print("duplicate entries ignored")
    session.rollback()
finally:
    session.close()

In [8]:
session = make_session()
try:
    print("#. rows in Programs table", session.query(Program).count())
except:
    session.rollback()
    raise
finally:
    session.close()

#. rows in Programs table 2


In [9]:
session = make_session()
try:
    session.add(program1_key_dupe())
    session.commit()
except sql.exc.IntegrityError:
    print("duplicate key ignored")
    session.rollback()
finally:
    session.close()

duplicate key ignored


In [10]:
# scoped session management
from contextlib import contextmanager

@contextmanager
def Session():
    """Provide a transactional scope around a series of operations."""
    print("-> creating session scope")
    session = make_session()
    try:
        yield session
        print("<- commiting session")
        session.commit()
    except:
        print("<- caught error in scope")
        session.rollback()
        raise
    finally:
        print("<- closing session in scope")
        session.close()

In [11]:
try:
    with Session() as session:
        print("#. rows in", Program.__tablename__, session.query(Program).count())
        session.add(program2())
except sql.exc.IntegrityError:
    print("integrity error")

-> creating session scope
#. rows in Programs 2
<- commiting session
<- caught error in scope
<- closing session in scope
integrity error


In [12]:
with Session() as session:
    for i, program in enumerate(session.query(Program).all()):
        print(i, program, program.contents)
    print("first program:", session.query(Program).first())

-> creating session scope
0 adc83b19e793491b1c6ea0fd8b46cd9f32e592fc int main() {}
1 f1d2d2f924e986ac86fdf7b36c94bcdf32beec15 int main(int argc, char **argv) {}
first program: adc83b19e793491b1c6ea0fd8b46cd9f32e592fc
<- commiting session
<- closing session in scope


In [13]:
def num_programs():
    with Session() as session:
        print("#. programs:", session.query(Program).count())

num_programs()
# remove a row
with Session() as session:
    session.query(Program).filter(Program.id == "adc83b19e793491b1c6ea0fd8b46cd9f32e592fc").delete()
num_programs()
with Session() as session:
    session.add(program1())
num_programs()

-> creating session scope
#. programs: 2
<- commiting session
<- closing session in scope
-> creating session scope
<- commiting session
<- closing session in scope
-> creating session scope
#. programs: 1
<- commiting session
<- closing session in scope
-> creating session scope
<- commiting session
<- closing session in scope
-> creating session scope
#. programs: 2
<- commiting session
<- closing session in scope


In [14]:
# add some devices
def devices():
    return [
        Device(hostname="host_A", platform="platform A", device="device A"),
        Device(hostname="host_A", platform="platform A", device="device B"),
        Device(hostname="host_B", platform="platform B", device="device B")
    ]

session = make_session()
try:
    with Session() as session:
        session.add_all(devices())
except sql.exc.IntegrityError:
    # if integrity error, check that entries exist
    with Session() as session:
        for d in devices():
            assert(session.query(Device).filter(
                Device.hostname == d.hostname,
                Device.platform == d.platform,
                Device.device == d.device).count() == 1)

-> creating session scope
<- commiting session
<- closing session in scope
